##  CLASSIFIER 학습

### 데이터는 어떻게 ?


- 일반적으로 이미지나 텍스트 데이터를 다룰 때 표준 Python 패키지를 이용하여 NumPy 배열로 불러오면 된다.
- 그 후 배열을 torch.*Tensor 로 변환한다.

- 이미지는 Pillow 패키지
- 텍스트의 경우 NLTK나 SpaCy가 유용 

### 이미지 분류 학습

1. torchvision 을 사용하여 CIFARR10의 train/test 데이터 셋을 불러오고, normalizing 한다. 
2. CNN 정의
3. loss function 정의
4. Train data를 통해 network를 학습시킨다.
5. test data를 사용해 network 검사한다. 


- torchvision을 사용하여 cifar10을 불러온다. 


In [1]:
import torch
import torchvision
import torchvision.transforms as transforms


In [2]:
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
# torchvision 데이터셋의 출력(output)은 [0, 1] 범위를 갖는 PILImage 이미지이다.
# 이를 [-1, 1]의 범위로 정규화된 Tensor로 변환


trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## CNN Model 

In [3]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

## Loss Function and Optimizer

In [4]:
import torch.optim as optim
# Cross-Entropy loss과 모멘텀(momentum) 값을 갖는 SGD
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## Training

In [5]:
for epoch in range(2):
    
    running_loss = 0.0
    for i, data in enumerate(trainloader,0):
        # [inputs, labels]의 목록인 data로부터 입력을 받음
        inputs, labels = data
        
        # gradient 매개변수를 0으로 setting
        optimizer.zero_grad()
        
        # forward - backward - optimize
        outputs = net(inputs)
        loss =  criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


[1,  2000] loss: 2.159
[1,  4000] loss: 1.799
[1,  6000] loss: 1.663
[1,  8000] loss: 1.580
[1, 10000] loss: 1.521
[1, 12000] loss: 1.469
[2,  2000] loss: 1.395
[2,  4000] loss: 1.372
[2,  6000] loss: 1.346
[2,  8000] loss: 1.334
[2, 10000] loss: 1.306
[2, 12000] loss: 1.284
Finished Training


## Test

In [6]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total +=labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 54 %
